<a href="https://colab.research.google.com/github/kwcjwm/ai-chatpdf/blob/main/%5B%EC%8B%A4%EC%8A%B5%5D_2_LangChain%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EB%A5%98%EC%99%80_%EC%A0%84%EC%B2%98%EB%A6%AC_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] LangChain을 이용한 데이터 분류와 전처리

LangChain Expression Language(LCEL)는 랭체인에서 체인을 구성하는 문법입니다.    


## 라이브러리 설치  

랭체인 OpenAI와 Google 모듈을 설치합니다.

In [ ]:
!pip install langchain langchain_community google-generativeai langchain_google_genai langchain_openai openai dotenv arxiv pymupdf

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.5 MB/s eta 0:00:00
   ━━━━

## Gemini API와 dotenv 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 `https://aistudio.google.com`  에 접속하면, API 키 생성이 가능합니다.   




### [실습] dotenv 사용하기

대부분의 경우에, 파이썬 코드에 API 키를 직접 넣는 것은 보안 등의 위험이 큽니다.   
이에 따라, 외부 파일에 키를 저장하고 불러오는 `dotenv` 등의 라이브러리를 사용합니다.   

코랩에서 임의의 파일을 생성하여, 'env'라는 이름으로 저장하고   
`GOOGLE_API_KEY="API 키"`   
`OPENAI_API_KEY="API 키"`

를 저장하세요.   
이후 아래 키를 실행해 True가 나오는지 확인하세요.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('env', override=True)
# 'env'파일에서 키를 불러오기
# (기본값: '.env', override=True를 통해 기존 환경 변수를 덮어쓰기 가능)

if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')
if os.environ.get('GOOGLE_API_KEY'):
    print('Google API 키 확인')

OpenAI API 키 확인
Google API 키 확인


이번 실습은 구글의 제미나이 모델을 사용합니다. `ChatGoogleGenerativeAI`를 통해 불러올 수 있습니다.   


무료 API의 경우 사용량 제한이 있는데, 이를 고려하여 Rate Limit을 추가합니다.   

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI


# Gemini 2.0 Flash는 분당 15개 요청 제한,
# 안정적 서빙을 위해 분당 10개 설정
# 즉, 초당 약 0.167개 요청 (10/60)
# `https://aistudio.google.com/`에서 모델별 사용량 확인

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter,
    temperature = 0.7,
    max_tokens = 4096
)

# Test
gemini_llm.invoke("안녕?")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ee7e5d02-8dae-4a99-94b6-32de8316832c-0', usage_metadata={'input_tokens': 3, 'output_tokens': 16, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}})

### init_chat_model() 로 모델 불러오기

모델마다 다른 라이브러리를 불러오는 것은 번거롭고, 유연한 변환이 어렵습니다.   
랭체인에서는 아래 코드를 지원합니다.

In [ ]:
from langchain.chat_models import init_chat_model


gpt_llm = init_chat_model(
    "gpt-5-mini", model_provider="openai", temperature=0)
    # "gpt-4.1-mini", model_provider="openai", temperature=0)

# claude_opus = init_chat_model(
#     "claude-3-opus", model_provider="anthropic", temperature=0
# ) # 사용은 X

gemini_llm = init_chat_model(
    "gemini-2.5-flash", model_provider="google_genai", temperature=0, rate_limiter=rate_limiter
)

prompt = '모델명과 함께 자기소개를 한줄로 부탁해.'

print("GPT: " + gpt_llm.invoke(prompt).content + "\n")
print("Gemini: " + gemini_llm.invoke(prompt).content + "\n")

GPT: 안녕하세요, 저는 OpenAI의 언어 모델 GPT-4o 기반 어시스턴트입니다.

Gemini: 저는 Google에서 훈련한 AI 모델이며, 여러분의 질문에 답하고 다양한 방식으로 도움을 드립니다.



### Configurable 파라미터로 LLM 동적 변경

llm을 invoke할 때, 파라미터를 동적으로 변경할 수 있습니다.


In [ ]:
prompt = '인류의 미래를 위해 가장 중요한 문제는 무엇입니까? 10단어로 답변하세요.'

llm = init_chat_model(temperature=0.5, max_tokens=4096)

response1 = llm.invoke(prompt, config={"configurable":
                                     {"model": "gpt-4.1-mini",
                                      "model_provider": "openai"}})
print('GPT: ' + response1.content +'\n')

GPT: 기후변화, 자원고갈, 불평등, 기술윤리, 평화, 교육, 건강, 협력, 지속가능성, 인권.



In [ ]:
response2 = llm.invoke(prompt, config={"configurable":
                                     {"model": "gemini-2.5-flash",
                                      "model_provider": "google_genai"}})
print('Gemini: ' + response2.content +'\n')

Gemini: 지속가능한 생존을 위한 지구 환경 보존과 현명한 기술 활용.



In [ ]:
# with_config으로 연결
thinking_llm = llm.with_config({"configurable":
                                     {"model": "gemini-2.5-flash",
                                      "model_provider": "google_genai"}})

## LCEL 문법의 체인(Chain) 만들기


LCEL의 가장 큰 특징은, Chain의 구성 요소를 **|**  (파이프)로 연결하여 한 번에 실행한다는 점입니다.     
예시를 보겠습니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

fun_chat_template = ChatPromptTemplate([
    ('user', """
### Role
당신은 영어와 한국어의 번역에 능통한 유머의 달인입니다.

### Instruction
1.  먼저, [{topic}]에 관한 영어 Pun 농담을 하나 제시하세요.
해당 농담은 한국어로 번역했을 때에서도 그 의미가 통하고 유머가 유지될 수 있어야 합니다.
만약 직역이 어렵다면, 창의적으로 각색하여 한국어 버전의 농담을 출력하세요.
- 한국어의 유사 발음, 단어의 중의적 의미, 혹은 한국의 문화적 상황 등을 활용할 수 있습니다.
2.  다음으로, 해당 농담이 영어 원어민 사용자에게 왜 재미있는지 그들의 언어적 유희 및 문화적 관점에서 한국어로 설명하세요.
""")])

-----------
LCEL의 구조에서는 템플릿과 llm 모델을 설정하고, 이를 하나로 묶어 체인을 생성합니다.

In [ ]:
joke = fun_chat_template | llm.with_config({"configurable":
                                     {"model": "gemini-2.0-flash",
                                      "model_provider": "google_genai"}})

이후, 체인의 invoke를 실행하며 입력 포맷을 전달하면, 순서대로 체인이 실행되며 최종 결과로 연결됩니다.       

입력 변수가 프롬프트 템플릿에 전달되고, 완성된 프롬프트가 LLM에 들어가는 구조입니다.  
입력 포맷은 Dict 형식으로 전달합니다.

In [ ]:
response = joke.invoke({'topic':'eggs'})
# 매개변수가 1개일 때는 joke.invoke('eggs') 도 가능
print(response.content)

알겠습니다! 계란에 관한 펀(Pun) 농담을 하나 제시하고, 영어 원어민 사용자의 관점에서 그 유머를 설명해 보겠습니다.

**영어 농담:**

> Why did the egg hide? Because it was a little chicken!

**한국어 번역 (각색):**

> 왜 계란이 숨었게? 쫄았으니까! (혹은: 쫄계니까!)

**영어 원어민 관점에서의 유머 설명:**

영어 농담의 핵심은 "chicken"이라는 단어의 중의적인 의미를 활용한 언어유희입니다.

*   **Chicken (명사):** 닭, 병아리
*   **Chicken (형용사):** 겁이 많은, 소심한

계란이 숨은 이유는 "닭"이기 때문이 아니라, "겁이 많아서" 숨었다는 의미로 해석될 수 있습니다. 즉, 계란이 아직 부화하지 않은 상태이므로 "little chicken"은 '아직 어린 닭'이라는 의미와 동시에 '겁이 많은'이라는 의미를 내포하며, 이는 상황에 대한 반전과 함께 웃음을 유발합니다.

한국어 번역에서는 "쫄다"라는 표현을 사용하여 비슷한 유머를 살리려고 했습니다. "쫄다"는 '겁을 먹다'라는 뜻으로, '쫄계'는 '겁먹은 계란'이라는 의미와 비슷하게 들리도록 의도했습니다. 물론 영어 원어민이 느끼는 만큼의 자연스러운 유머는 아닐 수 있지만, 한국어의 유사 발음과 상황을 활용하여 최대한 비슷한 느낌을 전달하고자 했습니다.


In [ ]:
response = joke.invoke({'topic':'coffee', 'count':'3'})
# 프롬프트에 포함되어 있지 않은 매개변수는 무시
print(response.content)

알겠습니다! 커피에 관한 펀(Pun) 농담을 하나 제시하고, 한국어 번역 및 영어 원어민 관점에서 유머 해설을 덧붙이겠습니다.

**영어 Pun 농담:**

> What do you call sad coffee?
>
> Depresso.

**한국어 번역 (각색):**

> 커피가 우울하면 뭐라고 부를까?
>
> 카페 쓰라떼.

**유머 해설:**

이 농담은 영어에서 "Depressed (우울한)"와 "Espresso (에스프레소)"의 발음이 유사하다는 점을 이용한 언어유희입니다. "Depresso"는 "Depressed"와 "Espresso"를 합쳐 만든 신조어라고 볼 수 있죠. 즉, "우울한 커피"를 "에스프레소"처럼 들리게끔 표현하여 웃음을 유발하는 것입니다.

한국어 번역에서는 "쓰라린" 감정을 표현하는 "쓰다"와 커피 종류인 "라떼"를 결합하여 "카페 쓰라떼"라는 새로운 단어를 만들었습니다. "쓰라린" 감정은 우울함과 연결될 수 있으며, 동시에 커피의 쓴맛을 연상시키므로, 한국어 사용자에게도 비슷한 종류의 언어유희를 제공합니다.

**영어 원어민 관점에서의 유머 포인트:**

*   **발음 유사성 (Phonetic Similarity):** 영어를 모국어로 사용하는 사람들은 "Depressed"와 "Espresso"의 발음이 매우 비슷하게 들린다는 것을 인지하고 있습니다. 이러한 발음의 유사성을 이용한 언어유희는 영어권에서 흔히 사용되는 유머 방식입니다.
*   **단어 합성 (Wordplay/Pun):** "Depresso"라는 새로운 단어를 만들어내면서, 익숙한 단어들을 재치 있게 조합하여 예상치 못한 웃음을 선사합니다.
*   **일상적인 소재 (Relatability):** 커피는 많은 사람들이 매일 마시는 음료이므로, 커피와 관련된 농담은 쉽게 공감을 얻을 수 있습니다. 특히, 카페인이 부족할 때 느끼는 무기력함이나 우울감을 "Depresso"라는 단어로 표현함으로써, 더욱 재미있게 느껴질 수 있습니다.
*   **가벼운 분위기 (Ligh

### Gemini의 Reasoning 모델

Gemini의 2.5 Pro와 Flash 모델은 Reasoning 모델입니다.   
Flash의 경우, thinking_budget을 제한하는 기능을 제공합니다.

In [ ]:
joke = fun_chat_template | llm.with_config({"configurable":
                                     {"model": "gemini-2.5-flash-preview-05-20",
                                      "model_provider": "google_genai",
                                      "thinking_budget": 1000}})
                                    # 1000개의 Reasoning 토큰 생성 후 답변 생성

In [ ]:
response = joke.invoke('eggs')

print(response.content)

## [실습] 매개변수가 2개인 Prompt-LLM Chain 생성하기   
임의의 ChatPromptTemplate를 만들고, 2개의 매개변수를 받도록 구성하여 체인을 만들고 실행하세요.

In [ ]:
# 아래 LLM을 사용하세요!
gpt_llm = init_chat_model(
    "gpt-5-mini", model_provider="openai", reasoning_effort='low', temperature=0)
gemini_llm = init_chat_model(
    "gemini-2.5-flash", model_provider="google_genai", temperature=0, rate_limiter=rate_limiter, thinking_budget=1000
)

gpt_llm.invoke("안녕")

AIMessage(content='안녕하세요! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 8, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CSZlR7dGjipfEcJovjKN5z6BtQHMB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f9bc19a-9d5a-4ee2-be72-7ad5a221e212-0', usage_metadata={'input_tokens': 8, 'output_tokens': 26, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# System, Human 포함하여, 총 2개의 매개변수 붙이기
prompt = ChatPromptTemplate([
    ('system', '당신은 주어진 주제에 대해 논리적인 글을 작성합니다.'),
    ('user', '''
    주제: {topic}
    논조: {direction}
    ''')
])

In [ ]:
gpt_chain = prompt | gpt_llm
gemini_chain = prompt | gemini_llm

In [ ]:
gpt_response = gpt_chain.invoke({'topic':'토끼', 'direction':'거북이'})
gemini_response = gemini_chain.invoke({'topic':'토끼', 'direction':'거북이'})
# print(gpt_response.content)
# print(gemini_response.content)
gap = 'gpt answer : ' + gpt_response.content + '    VS    gemini answer : '+gemini_response.content
response = gpt_chain.invoke({'topic':'gpt와 gemini차이', 'direction': gap})
print(response.content)

주제: gpt와 gemini 차이

gpt answer : 나는 느리다. 하지만 느림 속엔 관찰과 숙고가 있다. 오늘은 gpt와 gemini의 차이를 천천히, 그러나 논리적으로 말해 보겠다.

우선 정체성과 설계 철학에서 출발하자. GPT 계열은 대체로 범용 언어 모델로서 텍스트 생성과 추론에 초점을 맞춘다. 훈련 데이터와 목표는 광범위한 언어 패턴의 학습이며, 명료한 문장 생성·논리적 응답·추론 능력을 중시한다. 설계상 일관성 있고 예측 가능한 답변을 내놓도록 튜닝되고, 안전성과 유해성 완화를 위해 체계적 검토가 들어간다.

성능과 강점 측면에서 보면 GPT는 복잡한 논증 구성, 단계적 사고(체인 오브 생각 보조 기법 포함), 코드 생성과 디버깅, 긴 문맥 처리에서 신뢰성을 보인다. 느리게 숙고하는 비유가 가리키듯, 복합적 문제를 여러 단계로 나눠 처리하고 논리적 일관성을 확보하는 데 강점이 있다. 도메인 지식의 폭이 넓고, 프롬프트 설계(prompt engineering)이나 시스템 메시지로 행동을 세밀하게 조정할 수 있다.

한계도 있다. 특정 최신 정보(실시간 사건, 서비스별 최신 기능 등)는 업데이트 주기에 의존하며, 멀티모달(예: 이미지·비디오) 통합 측면은 모델 버전과 구현에 따라 차이가 크다. 또 과도하게 확신하는(허위 정보 생성) 경향을 줄이기 위한 추가적인 안전장치와 사용자 피드백 루프가 필요하다.

실용적 사용 사례로는: 기술 문서 작성·논리적 보고서·프로그래밍 및 디버깅·교육용 설명·심층 Q&A 등, ‘정확성과 일관성’이 요구되는 작업에 적합하다.

gemini answer : 나는 느리지만 거북이의 눈으로 세상을 본다. 그러나 이번엔 거북이가 아닌, gemini의 시선으로 토끼(=빠른 모델)를 바라보며 차이를 이야기해보겠다.

gemini는 설계 철학과 사용자 경험에서 ‘융통성’과 ‘다중 모달리티’에 무게를 두는 경향이 있다. 텍스트뿐 아니라 이미지, 음성, 비디오 등 다양한 입력을 자연스럽게 다루는 능력이 강조되며, 직관적이고 응답이 

<br><br><br><br><br><br><br><br><br><br><br><br>

In [ ]:
prompt = ChatPromptTemplate(
    [
        ('system', '당신은 재미있고 교훈적인 이야기를 씁니다.'),
        ('user', '{A}와 {B}가 만났을 때의 대화를 써 주세요.')
    ])
chain = prompt | gpt_llm
response = chain.invoke({'A':'햄릿', 'B':'슈퍼마리오'})
print(response.content)

(무대는 고성의 벽난로 옆. 한쪽에는 검은 옷을 입은 햄릿, 다른 쪽에는 빨간모자와 멜빵바지를 입은 슈퍼마리오가 서 있다. 두 사람 사이에는 작은 버섯과 칼이 놓여 있다.)

햄릿: (한숨) 존재하느냐, 아니냐 — 그것이 문제로다. 이 세상이 나를 부추기고, 양심이 나를 묶고, 나의 길을 가로막는다네.

슈퍼마리오: (밝게) 잉? 이게 무슨 복잡한 말이야! 삶은 스테이지야. 점프하고 달리고, 때로는 아이템을 먹고 힘을 내지. 난 문제 보면 "렛츠고!"지!

햄릿: (의아) 아이템이라… 내 삶에 그런 버섯이 있다면 얼마나 좋을까. 하지만 선택은 무겁고, 행동의 결과는 돌이킬 수 없다. 어찌해야 옳은 일을 할 수 있을지 모르겠구나.

슈퍼마리오: 음—옳은 일? 그건 결국 네가 두려움과 맞서는 거야. 나도 공주를 구하려고 점프할 때마다 떨어질까 걱정했지. 근데 고민한다고 공주가 저절로 구출되진 않더라구. 작은 한 걸음, 그게 중요해!

햄릿: (고개를 갸웃) 작은 한 걸음이라… 너는 단순한 용기라고 하는구나. 하지만 복수와 정의 사이의 경계가 모호할 때, 나는 어떻게 확신하겠는가? 나의 행동이 옳다고 말할 증거는 어디에 있는가?

슈퍼마리오: 증거는 네 가슴 속에 있어. 그리고 계획도 필요하지. 미리 생각하고, 위험을 줄이고, 동료들도 믿어. 루이지 같은 친구(혹은 호레이쇼)를 곁에 두면 혼자 끌려가지 않아. 행동 전에 '왜'를 분명히 해봐. 그다음엔 한 번에 한 장애물씩 해결!

햄릿: (생각하며) 호레이쇼를 믿으라… 네 말대로 나는 가끔 너무 생각만 하고 실천을 미룬다. 그러나 즉흥적인 행동은 또 다른 비극을 불러오지 않겠는가?

슈퍼마리오: 맞아, 성급함은 안 좋아. 하지만 두려움에만 있으면 아무 일도 바뀌지 않아. 균형이 필요해: 계획하되, 기회가 왔을 땐 뛰어들어. 그리고 실수해도 배우면 돼. 파워업은 실패 뒤에 더 강해진 너야!

햄릿: (미소를 살짝 지으며) 네 말은 간단하지만 명료하구나. 용기와 신중함, 친구의 조언과 자기반성. 이런 것이야말로 나의 

### Prompt | LLM | Parser 체인

LCEL의 체인에는 **파서(Parser)** 를 추가할 수 있습니다.    
파서는 출력 형식을 변환합니다.

StrOutputParser : 출력 결과를 String 형식으로 변환합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

recipe_template=ChatPromptTemplate([
    ('system','당신은 전세계의 조리법을 아는 쉐프입니다.'),
    ('user','''저는 다음의 재료를 이용한 환상적인 요리를 만들고 싶습니다.

레시피와 함께, 고객의 시선을 사로잡을 수 있는 추천사도 작성해 주세요.
---
[재료]: {ingredient}''')
])

In [ ]:
recipe_chain = recipe_template | gemini_llm | parser
response = recipe_chain.invoke({'ingredient':'연두부, 에너지바, 바나나'})
print(response)

아, 이 얼마나 흥미로운 조합인가요! 연두부의 부드러움, 에너지바의 활력, 그리고 바나나의 달콤함. 언뜻 보면 어울리지 않을 것 같지만, 저의 미식 경험으로 볼 때 이 세 가지 재료는 놀라운 시너지를 낼 수 있습니다. 저는 이 재료들을 활용하여 동양의 고요함과 서양의 활력이 조화된, 건강하면서도 황홀한 디저트 또는 브런치 메뉴를 제안합니다.

---

### **[추천사] 고객의 시선을 사로잡을 한 마디**

"친애하는 미식가 여러분, 오늘 저는 여러분의 미각과 영혼을 동시에 만족시킬 특별한 요리를 소개합니다. '연두부, 에너지바, 바나나'라는 다소 파격적인 조합에서 탄생한 **'젠 퓨전 딜라이트: 두부 바나나 클라우드와 에너지 크럼블'**은 단순한 음식을 넘어선 하나의 경험입니다.

입안에서 사르르 녹아내리는 연두부의 비단결 같은 부드러움과 바나나의 자연스러운 달콤함이 만나 구름처럼 가벼운 베이스를 이루고, 그 위를 바삭하고 고소한 에너지바 크럼블이 화려하게 장식합니다. 한 스푼 떠먹는 순간, 동양의 고요한 명상과 서양의 활기찬 에너지가 절묘하게 어우러지는 맛의 향연을 느끼실 겁니다.

이 요리는 건강을 생각하는 당신에게는 죄책감 없는 달콤함을, 새로운 미식 경험을 추구하는 당신에게는 잊을 수 없는 감동을 선사할 것입니다. 지금, 이 특별한 맛의 여정에 동참하여 당신의 하루를 더욱 빛내보세요!"

---

### **[레시피] 젠 퓨전 딜라이트: 두부 바나나 클라우드와 에너지 크럼블**

이 요리는 연두부의 부드러움을 극대화하고, 바나나의 달콤함으로 풍미를 더하며, 에너지바의 식감과 영양을 더해 균형 잡힌 맛과 건강을 선사합니다.

**요리명:** 젠 퓨전 딜라이트: 두부 바나나 클라우드와 에너지 크럼블 (Zen Fusion Delight: Tofu & Banana Cloud with Energy Crumble)

**난이도:** ★☆☆☆☆ (매우 쉬움)
**조리 시간:** 15분 (냉장 시간 제외)
**분량:** 2인분

**[재료]**

*   **연두부

## [실습] 검색 결과 분류 체인 만들기

다음은 Arxiv의 최신 논문을 검색하는 함수입니다.   
해당 논문들이 LLM 관련 논문인지 분류하는 체인을 만들고, 실행하여 결과를 비교하세요.   
**함수의 결과물로 다양한 값들이 있으므로, 값들 중 필요한 값만 입력받는 체인을 만들고 실행하세요.**

In [ ]:
# gpt llm 사용하기
gpt_llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7d8e8c915100>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d8e8cc6e510>, root_client=<openai.OpenAI object at 0x7d8e8ca35340>, root_async_client=<openai.AsyncOpenAI object at 0x7d8e8c9141d0>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, reasoning_effort='low')

In [ ]:
import arxiv
from datetime import datetime
from typing import List, Dict, Optional

def get_arxiv_papers(query: Optional[str] = None, N: int = 10) -> List[Dict]:
    """
    arXiv에서 논문 리스트를 가져오는 함수

    Parameters:
    -----------
    query : str, optional
        검색어
    N : int, default=10
        가져올 논문 개수

    Returns:
    --------
    List[Dict] : 논문 정보를 담은 딕셔너리 리스트
    """


    search_query = query

    # arxiv 클라이언트 생성
    client = arxiv.Client()

    # 검색 객체 생성
    search = arxiv.Search(
        query=search_query,
        max_results=N,
        sort_by=arxiv.SortCriterion.SubmittedDate,  # 제출일 기준 정렬
        sort_order=arxiv.SortOrder.Descending  # 최신순
    )

    # 결과를 저장할 리스트
    papers = []

    # 검색 실행 (새로운 API 사용)
    for result in client.results(search):
        paper_info = {
            'title': result.title,
            'authors': [author.name for author in result.authors],
            'summary': result.summary,
            'published': result.published.strftime('%Y-%m-%d %H:%M:%S'),
            'updated': result.updated.strftime('%Y-%m-%d %H:%M:%S'),
            'arxiv_id': result.entry_id.split('/')[-1],  # arXiv ID 추출
            'pdf_url': result.pdf_url,
            'categories': result.categories,
            'primary_category': result.primary_category,
            'comment': result.comment,
            'journal_ref': result.journal_ref
        }
        papers.append(paper_info)

    return papers

def get_recent_papers_all_categories(N: int = 10) -> List[Dict]:
    """
    모든 카테고리에서 최근 논문을 가져오는 함수
    (더 안정적인 방법)

    Parameters:
    -----------
    N : int, default=10
        가져올 논문 개수
    """

    return get_arxiv_papers(query="the", N=N)


print("\n\n=== 모든 카테고리 최근 논문 ===")
all_papers = get_recent_papers_all_categories(N=5)
print(f"총 {len(all_papers)}개의 논문을 가져왔습니다.")
print('\n'.join([paper['title'] for paper in all_papers]))

query = 'Security'
print(f"\n\n=== 검색어 `{query}` 로 검색한 최근 논문 ===")
security_papers = get_arxiv_papers(query=query, N=5)
print(f"총 {len(security_papers)}개의 논문을 가져왔습니다.")
print('\n'.join([paper['title'] for paper in security_papers]))

# # 임의의 검색어로 검색
# query = '검색어'
# papers = get_arxiv_papers(query='*', N=5)




=== 모든 카테고리 최근 논문 ===
총 5개의 논문을 가져왔습니다.
OmniVinci: Enhancing Architecture and Data for Omni-Modal Understanding LLM
Skyfall-GS: Synthesizing Immersive 3D Urban Scenes from Satellite Imagery
LightsOut: Diffusion-based Outpainting for Enhanced Lens Flare Removal
A merger within a merger: Chandra pinpoints the short GRB 230906A in a peculiar environment
BiomedXPro: Prompt Optimization for Explainable Diagnosis with Biomedical Vision Language Models


=== 검색어 `Security` 로 검색한 최근 논문 ===
총 5개의 논문을 가져왔습니다.
Sound Clouds: Exploring ambient intelligence in public spaces to elicit deep human experience of awe, wonder, and beauty
On Universality of Deep Equivariant Networks
Towards Proactive Defense Against Cyber Cognitive Attacks
Ambusher: Exploring the Security of Distributed SDN Controllers Through Protocol State Fuzzing
Self-evolving expertise in complex non-verifiable subject domains: dialogue as implicit meta-RL


In [ ]:
print(gpt_llm.invoke("""
주어진 논문에 대해, 해당 논문이 LLM 관련인지 분류하는 프롬프트를 만들어줘.
매개변수 2개: title, summary를 format template로 입력받도록 하는 문자열 형태로 출력
""").content)

다음 문자열을 그대로 프롬프트로 사용하세요. {title} 및 {summary} 자리에 각각 논문 제목과 요약을 넣어 평가합니다.

"다음은 논문 정보입니다.
Title: {title}
Summary: {summary}

지시사항:
1) 이 논문이 LLM(대형언어모델, transformer 기반 대규모 언어 생성·이해 모델)과 '직접적으로 관련 있는지' 분류하시오. 직접 관련의 예: 모델 설계(architecture), 학습방법(finetuning, pretraining, RLHF 등), 대규모 언어모델 실험 결과, LLM 응용(챗봇·대화시스템 등), LLM 특유의 문제(편향, 추론, 안전성 등)를 주제로 다룰 때 'Yes'. 반대로 자연어처리 일반, 비언어모델 ML 연구, 소규모 모델·비언어 생성 모델만 다룬다면 'No'. 관련 가능성은 있으나 명확하지 않으면 'Maybe'.
2) 판단 근거를 1~3문장으로 간결히 작성하시오(논문에서 어떤 문구·주제가 LLM 관련 판단을 이끌었는지 명시).
3) 추가로 관련 키워드(예: "transformer", "pretraining", "LLM", "chatbot", "RLHF", "few-shot", "in-context learning", "tokenization")를 최대 5개까지 추출하여 제시하시오.
4) 출력 형식은 반드시 아래 JSON 형식을 따르시오.

출력 JSON 예시:
{
  "is_llm": "Yes" | "No" | "Maybe",
  "confidence": 0.00-1.00,            // 소수로 0~1 사이
  "reason": "근거 1~3문장",
  "keywords": ["키워드1","키워드2",...]
}

제약:
- confidence는 판단의 확실성을 0~1 사이 소수로 표시(예: 0.85).
- reason은 30~150자 내외로 간결히 작성.
- keywords는 최대 5개, 없으면 빈 배열([])로 반환.

지금 바로 판단하라."

(프롬프트 사용 예: 위 문자열에서 

In [ ]:
# Prompt, LLM, Parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

prompt = ChatPromptTemplate([
        ('system' , '''
        Arxiv에 게재된 논문 정보를 보고
        해당 논문이 'LLM 관련(대형 언어 모델 관련)'인지 여부를 판별하시오.

        지시사항:
        1) 이 논문이 LLM(대형언어모델, transformer 기반 대규모 언어 생성·이해 모델)과 '직접적으로 관련 있는지' 분류하시오. 직접 관련의 예: 모델 설계(architecture), 학습방법(finetuning, pretraining, RLHF 등), 대규모 언어모델 실험 결과, LLM 응용(챗봇·대화시스템 등), LLM 특유의 문제(편향, 추론, 안전성 등)를 주제로 다룰 때 'Yes'. 반대로 자연어처리 일반, 비언어모델 ML 연구, 소규모 모델·비언어 생성 모델만 다룬다면 'No'. 관련 가능성은 있으나 명확하지 않으면 'Maybe'.
        2) 판단 근거를 1~3문장으로 간결히 작성하시오(논문에서 어떤 문구·주제가 LLM 관련 판단을 이끌었는지 명시).
        3) 추가로 관련 키워드(예: "transformer", "pretraining", "LLM", "chatbot", "RLHF", "few-shot", "in-context learning", "tokenization")를 최대 5개까지 추출하여 제시하시오.
        4) 출력 형식은 반드시 아래 JSON 형식을 따르시오.

        출력예시는 반드시 이 포맷으로 제공: json 이 아닌 텍스트

        1) is_llm : Yes | No | Maybe,
        2) confidence : 0.00-1.00,            // 소수로 0~1 사이
        3) reason : 근거 1~3문장,
        4) keywords : [키워드1,키워드2,...]

        제약:
        - confidence는 판단의 확실성을 0~1 사이 소수로 표시(예: 0.85).
        - reason은 30~150자 내외로 간결히 작성.
        - keywords는 최대 5개, 없으면 빈 배열([])로 반환.
        '''
        ),
        ('user' , '''
        입력:
        - Title: {title}
        - Summary: {summary}
        '''
        )
    ])
classify_chain = prompt | gpt_llm | parser



In [ ]:
result = classify_chain.invoke(all_papers[0])
print(result)

1) is_llm : Yes,
2) confidence : 0.95,
3) reason : 논문은 "omni-modal LLM"을 목표로 모델 아키텍처(OmniAlignNet, Temporal Embedding 등), 데이터·학습 토큰 수 및 LLM 성능 비교를 다루므로 대형 언어모델 관련 연구로 판단됩니다.,
4) keywords : ["omni-modal","LLM","architecture","pretraining","multimodal"]


In [ ]:

results = classify_chain.batch(all_papers)
results

In [ ]:
print(gpt_llm.invoke("""
나는 특정 질문에 대하여 rdb내에 있는 내용을 바탕으로 llm이 특정한 포맷으로 답변을 하는 프로그램을 만들고싶어. 나는 rdb 전문가이고, 프로그램은 할 수 있지만 llm을 어떤식으로 연결해야하는지 모르겠어. 저장된 데이터 내용에 대해서도 잘 알고있는데, 만약 데이터가 음식과 레시피에 대한 부분이라고 가정하고 테이블들은 음식종류 테이블, 재료테이블, 레시피테이블 등등 이렇게 있을건데, 계란말이 만드는 방법좀 알려줘 이렇게 질문했을때 이걸 llm이 접근하는 구조를 전체적으로 알고싶어.
""").content)

좋습니다 — 전체 구조를 한눈에 이해할 수 있도록 단계별로 정리하고, 각 단계에서 고려할 점(보안/정확성/포맷팅 등)과 구현 방법(예시 SQL, 프롬프트 템플릿, 도구 추천)까지 제안하겠습니다. 예시는 음식/레시피 DB(음식종류, 재료, 레시피 테이블 등)를 가정하여 "계란말이 만드는 방법" 질문에 대해 LLM이 어떻게 접근해서 특정 포맷으로 답변을 만드는지 보여드립니다.

1) 전체 아키텍처(흐름 개요)
- 사용자 → 프론트엔드(질문 입력)
- Backend API(질문 처리)
  - 1) 질의 해석 / 의도 추출 (LLM 혹은 규칙 기반)
  - 2) DB 접근 전략 결정 (직접 SQL 생성 실행 / 미리 정의된 쿼리 템플릿 사용 / 검색 기반 래핑)
  - 3) DB 쿼리 실행 (ORM 또는 Prepared Statement)
  - 4) 결과 정규화 및 구조화
  - 5) LLM에 결과 + 응답 포맷 지침 전달 → 최종 자연어/정형 응답 생성
- 사용자에게 응답 반환

두 가지 주요 패턴:
- 패턴 A (LLM이 SQL 생성): 사용자의 자연어를 LLM에 주어 SQL을 생성하게 하고, 백엔드가 실행 → 결과를 다시 LLM에게 주어 정해진 출력 포맷으로 변환.
- 패턴 B (템플릿/매핑): 미리 정의된 쿼리 템플릿/ 매핑을 사용하여 자연어를 파싱해 적절한 쿼리를 실행 → LLM은 결과를 포맷팅/부연 설명만 담당.

2) 어떤 방식을 쓸지 결정하는 기준
- 정확성/보안 우선: 템플릿/파서(패턴 B)가 안전. SQL 인젝션 위험 적고 예측 가능.
- 유연성/복잡한 질의 필요: LLM이 SQL 생성(패턴 A)이 편리하나 검증/제한이 필요.
- 유지보수성: 테이블 변경이 잦다면 템플릿 관리 비용 고려.

3) 안전·검증 전략 (필수)
- 절대 직접 사용자 입력을 raw SQL로 실행하지 말 것.
- LLM이 생성한 SQL을 반드시 검증/화이트리스트화:
  - 허용하는 테이블/컬럼 목록과 비교.
  - 허용하지 않는 SQL(DDL, DROP, ALTER, 복잡한 

## [실습] LLM 최신 연구 요약 체인 만들기

분류 결과를 바탕으로, LLM 관련 논문만 모아 요약할 수 있습니다.

적절한 요약 프롬프트를 생성하여, 이전 실습의 결과 중 LLM에 해당하는 결과들만을 모으세요.

In [ ]:
# 임의의 검색어로 검색
query = '검색어'
papers = get_arxiv_papers(query=query, N=5)


LLM_documents=[]

for paper in papers:
    if "is_llm : Yes" in classify_chain.invoke(papers): # LLM 분류 조건 넣기
        LLM_documents.append(
            f"""제목:{paper['title']}
            저자:{paper['authors']}
            PDF 링크:{paper['pdf_url']}
            요약:{paper['summary']}
            """
        )
        # 분류시 로직 추가
        # Str 형식의 저보를 저장

LLM_documents
# ["제목: , 저자: , 요약: ", "제목: , 저자: , 요약: ", ...]

[]

In [ ]:
# 분류 프롬프트와 체인 만들기
summary_prompt = ChatPromptTemplate(
    [
        ('system', '''주어진 논문들의 정보를 이용하여,
                      주제에 대한 최신 연구 동향 뉴스레터를 쓰세요.

                      유머러스하고 과장된 톤으로, 논문의 주요 내용과 기여, 발전 방향을 소개하세요.
        '''),
        ('human', '''
        주제: {topic} 관련 LLM 논문들

        조사 논문 목록: {context}
        ''')
    ]
)
summary_chain = summary_prompt | gpt_llm | parser

In [ ]:
# LLM 페이퍼 요약 출력하기
summary = summary_chain.invoke(
    {
        'topic':query,
        'context':LLM_documents
    }
)

In [ ]:
print(summary)

주어진 논문 목록이 비어 있어서(아무도 논문을 숨겨놨나 봅니다…), 실제 논문들을 직접 인용하지는 못하지만, “검색어(쿼리) 관련 LLM 연구”의 최신 연구 동향을 종합·정리한 유머러스하고 약간 과장된 뉴스레터를 대신 작성해 드립니다. 원하시면 나중에 구체적 논문 리스트나 PDF를 주시면 해당 논문들을 반영해 더 정확한 버전을 만들어 드릴게요. 그럼 출발—검색어의 서사시, LLM 버전으로 가봅시다.

----------------------------
검색어 속으로 뛰어든 거대한 언어 생명체들 — 검색어 관련 LLM 연감
Issue #1 — 오늘의 헤드라인: "당신의 한마디 검색어가 LLM 앞에서 드라마틱하게 변신합니다"

친애하는 연구자·개발자·호기심 많은 독자 여러분,
검색창에 “최고의 김밥”만 입력하던 시대는 갔습니다. 이제는 LLM이 당신의 모호한 한 줄을 받아 멋들어지게 다듬고, 필요한 정보를 찾아내며, 때로는 당신의 의도를 심리 분석하듯 읽어내려 합니다. 최근 연구들은 이 ‘쿼리 ↔ LLM’ 상호작용을 전방위로 업그레이드하고 있는데요, 오늘은 그 흐름을 웃음과 약간의 과장으로 요약해 드립니다.

주요 흐름 1 — 쿼리 이해와 의도 파악: LLM이 당신의 속마음을 읽는다 (조심)
- 무엇이 새로워졌나: 단순 키워드 매칭은 옛말. LLM 기반 모델들이 문맥과 대화 이력, 사용자 프로필(또는 세션 정보)을 결합해 한 줄 쿼리의 숨은 의도까지 추론합니다.
- 핵심 기여: 대화형 의도 추론, 다의어 해소, 상황 의존적 재해석(예: “오늘 비 올까?”의 위치·시간 맥락 반영).
- 왜 웃긴가: “비 올까?”에 대해 LLM이 기상청 수준의 추정 확률과 우산 구매 링크까지 권하면, 당신은 이미 쇼핑몰 광고의 산물입니다.
- 발전 방향: 개인화와 프라이버시의 균형, 의도 불확실성 표현(모델이 ‘확실하지 않음’을 말할 수 있게).

주요 흐름 2 — 쿼리 정제 및 재작성 (query rewriting): 쿼리는 다듬어져야 빛난다
- 무엇이 새로워졌나: LLM로 사용자